# Exam Part B

Here we will explain our thinking for the question B of the Exam.

# Question B1

We can now calculate some reference values for the **2D Gas**! In our units we have:

$$
\lambda = \frac1{r_s^2}
$$

First we calculate the Fermi wave vector $k_F$ where $N_k$ is the total number of k-states inside the Fermi surface and $V_k$ is the volume occupied by each k-state (using 2D volumes):   
$$
\begin{align*}

V_F = \pi k_F^2 &\hArr N_k V_k = \pi k_F^2
\\\\

&\hArr k_F^2 = \frac{N_k \cdot \frac{4\pi^2}{V}}{\pi}
\\\\

&\hArr k_F^2 = \frac{4\pi N_k}{V}

\end{align*}
$$

Then the energy (assuming two electrons in each state):

$$
\begin{align*}

E &= 2\lambda \cdot \sum_{|\textbf k| \le k_F} k_j^2
\\\\

&= 2\lambda \cdot \frac{V}{(2\pi)^2} \int_0^{k_F}\int_0^{2\pi}k^2\cdot k\ d\theta\ dk
\\\\

&= \lambda \frac V\pi \cdot \left[\frac14k^4\right]_0^{k_F}

\end{align*}
$$

If the total number of electrons is $N$ we can write, in the **thermodynamic limit**:
- **Unpolarized gas**: In this case we have that $N = 2N_k$ and as such:
    $$
    \begin{align*}
    k_F^2 = 2\pi n
    \end{align*}
    $$
        
    Meaning (remembering density is a constant equal to $1/\pi)$:
    $$
    \begin{align*}
    \frac EN &= \lambda \cdot \frac VN \cdot \pi n^2
    \\\\
    &= \lambda
    \end{align*}
    $$
    
- **Polarized gas**: In this case we have that $N = N_k$ and as such:
    
    $$
    \begin{align*}
    k_F^2 = 4\pi n
    \end{align*}
    $$
    
    Meaning (we have to divide the energy by 2 because now there is only one electron in each k-state):
    
    $$
    \begin{align*}
    \frac EN &= \frac12 \cdot 4\lambda \cdot \frac VN \cdot \pi n^2
    \\\\
    &= 2\lambda
    \end{align*}
    $$

Now we can calculate the expected value for $N = 21$ (I drew a simple grid on paper to see the distances and multiplicity) and we see a value close to $2$ as expected!

In [10]:
import numpy as np

# Parameters
N = 21
W = np.array([1, 4, 4, 4, 8])
D = np.array([0, 1, (1+1)**0.5, 2, (4+1)**0.5])
rs = 1

# V = N * pi * rs**2 therefore L = (N * pi)**0.5 * rs
L = np.sqrt(N * np.pi) * rs

# E in units of lambda
Epol = (np.sum(W * D**2) * (2 * np.pi / L)**2) / N

print(f"Energy per particle for polarized gas with rs = {rs} is E = {Epol:.3f} (in units of lambda)")

Energy per particle for polarized gas with rs = 1 is E = 1.938 (in units of lambda)


# Question B2

We used `polrs5` for the ID of the simulation.

---

We used the following code (in `polrs5.in`) to optimize the wave function:
```
vmc 20 500 1.0 20
optimize 500 polrs5.u3 polrs5.b
```

The optimization process indicated that the variational parameters converged! Then we ran the command `vmc 20 500 1.0 20` and got the results in file `polrs5.outB2` where the result for the last block was:
```
 ===>> vmc block           20
 -0.29204831908E+00 0.500E+03 -0.29200970870E+00 0.18E-03  elocal              
  0.52769122115E+00 0.500E+03  0.54007646940E+00 0.38E-02  acc.rate            
 -0.38559772710E+00 0.500E+03 -0.38630283138E+00 0.37E-03  epot                
  0.93549408024E-01 0.500E+03  0.94293122677E-01 0.50E-03  ekin                
  0.85458683021E-01 0.500E+03  0.85352788475E-01 0.11E-03  e2
```

We see that the acceptance rate is about 50% as desired.

# Question B3

We used the following command to run the DMC simmulation (we use `0.1` for the time step which is about one tenth of the time step used in VMC and use the VMC energy as the trial energy `-0.292` for our DMC):
```
dmc 20 50 0.1 50 -0.292
```

We got the results in file `polrs5.outB3` where the result for the last block was:
```
 ===>> dmc block           20
 -0.29255323809E+00 0.249E+04 -0.29297261163E+00 0.10E-03  elocal              
  0.96399231351E+00 0.249E+04  0.97053988588E+00 0.61E-03  acc.rate            
 -0.38593614286E+00 0.249E+04 -0.38512068292E+00 0.33E-03  epot                
  0.93382904773E-01 0.249E+04  0.92148071284E-01 0.39E-03  ekin                
  0.85682558702E-01 0.249E+04  0.85929193791E-01 0.63E-04  e2  
```

We expect to see a dependency of the energy on:
- The time step, because DMC is only exact in the zero time step limit, meaning that a finite time step introduces errors.
- The number of walkers, because our DMC simulation controls the population of walkers by adjusting the potential, which introduces a bias. This bias vanishes when the number of walkers increases to infinity because the variance on the number of walkers will go to zero (thanks to the central limit theorem).

Finaly we calculate the extrapolated estimator:

In [11]:
# From DMC
Emixed = -0.29297261163

# From VMC
Evar = -0.29200970870

# Extrapolated energy
print(f"Extrapolated Energy = {2 * Emixed - Evar:.4f}")

Extrapolated Energy = -0.2939


# Question B4

In [13]:
# Parameter
N = 21

# From Ref [1]
b1 = 0.0442
b2 = 0.0465

# From B1
rs = 5.0
DeltaTN = (2 - Epol) * (1/rs**2)

# From B3
EN = -0.2939

Einfty = EN - b1 * DeltaTN - b2 * (1/N)
print(f"Energy per particle in the thermodynamic limit E = {Einfty:.4f}")

Energy per particle in the thermodynamic limit E = -0.2962


# Question B5

In [14]:
def EHF(xi):
    return (1/rs**2) * ((1+xi)**2 + (1-xi)**2) - (4 * 2**0.5) / (3 * np.pi * rs) * ((1+xi)**(3/2) + (1-xi)**(3/2))


Ec0 = Einfty - EHF(0)
Ec1 = Einfty - EHF(1)

print(f"Correlation energy at xi = 0: {Ec0:.4f}")
print(f"Correlation energy at xi = 1: {Ec1:.4f}")

Correlation energy at xi = 0: -0.1361
Correlation energy at xi = 1: -0.1167
